In [1]:
#!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
#!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [5]:
#!gzip -dc fhvhv_tripdata_2021-01.csv.gz

In [6]:
#!wc -l fhvhv_tripdata_2021-01.csv

In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

In [8]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
#!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
!pip install pandas

In [11]:
import pandas as pd

In [12]:
#df_pandas = pd.read_csv('head.csv')

In [13]:
#df_pandas.dtypes

In [14]:
#spark.createDataFrame(df_pandas).schema

Integer - 4 bytes
Long - 8 bytes

In [15]:
from pyspark.sql import types

In [16]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [17]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

In [18]:
df = df.repartition(24)

In [19]:
df.write.mode('overwrite').parquet('fhvhv/2021/01/')

In [20]:
df = spark.read.parquet('fhvhv/2021/01/')

In [21]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [22]:
from pyspark.sql import functions as F

In [23]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02617|2021-01-21 18:19:13|2021-01-21 19:14:27|         256|         265|   NULL|
|           HV0003|              B02871|2021-01-26 11:15:51|2021-01-26 11:23:14|         165|         165|   NULL|
|           HV0003|              B02764|2021-01-19 19:11:47|2021-01-19 19:29:45|          82|         137|   NULL|
|           HV0003|              B02872|2021-01-27 11:49:13|2021-01-27 11:53:15|          97|          97|   NULL|
|           HV0005|              B02510|2021-01-27 15:21:49|2021-01-27 15:29:59|         134|          95|   NULL|
|           HV0003|              B02765|2021-01-17 18:35:49|2021-01-17 18:55:43|

In [24]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [25]:
crazy_stuff('B02884')

's/b44'

In [26]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [30]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()
    #.withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-21|  2021-01-21|         256|         265|
| 2021-01-26|  2021-01-26|         165|         165|
| 2021-01-19|  2021-01-19|          82|         137|
| 2021-01-27|  2021-01-27|          97|          97|
| 2021-01-27|  2021-01-27|         134|          95|
| 2021-01-17|  2021-01-17|          74|         116|
| 2021-01-28|  2021-01-28|          78|          75|
| 2021-01-04|  2021-01-04|         236|         126|
| 2021-01-21|  2021-01-21|          79|         137|
| 2021-01-29|  2021-01-29|         198|         157|
| 2021-01-24|  2021-01-24|          37|          37|
| 2021-01-05|  2021-01-05|         106|          72|
| 2021-01-20|  2021-01-20|          35|         265|
| 2021-01-10|  2021-01-10|          39|          39|
| 2021-01-26|  2021-01-26|          75|         265|
| 2021-01-06|  2021-01-06|         149|       

In [31]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [ ]:
#!head -n 10 head.csv